In [1]:
import numpy as np
from sklearn.manifold import TSNE
import pickle
# import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
import time
import sqlite3
import pandas as pd
import json

In [79]:
song_id_path = "./data/song_track_artist.pkl"

# Get embedding latent vectors

In [80]:
genre_embeddings_path = "./data/latent_embeddings.npy"
track_id_embeddings_path = "./data/track_ids_nn.txt"

In [81]:
genre_embeddings_np.shape

NameError: name 'genre_embeddings_np' is not defined

In [82]:
len(list(range(genre_embeddings_np.shape[1])))

NameError: name 'genre_embeddings_np' is not defined

In [94]:
genre_embeddings_np = np.load(genre_embeddings_path)
track_ids = np.loadtxt(track_id_embeddings_path, dtype="str")
genre_embeddings = pd.DataFrame(genre_embeddings_np, columns=list(range(genre_embeddings_np.shape[1])))
genre_embeddings.index = track_ids

In [97]:
genre_embeddings.shape

(5488, 50)

# Get song factors

In [83]:
with open("./data/song_factors.pkl", "rb") as f:
    song_id_factors_dict = pickle.load(f)

In [84]:
song_id_factors = pd.DataFrame(song_id_factors_dict).T

# get song popularity

In [85]:
with open("./data/popularity.pkl", "rb") as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    song_popularity = u.load()


# get genre

In [86]:
with open("./data/track-genre-mapping.json", "r") as f:
    track_genre = json.load(f)

# Set song id filter to filtered_song_ids (used throughout)

In [87]:
filtered_song_ids = song_popularity.index[:3000]

# Get song_id to song_name mapping


### using pkl file

In [88]:
with open(song_id_path, "rb") as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    song_name_artist = u.load()

In [89]:
# getting everything in the right order
song_name_artist_ordered = song_name_artist.set_index(["song_id"]).loc[filtered_song_ids]
# song_artist_str = song_name_artist_ordered.apply(lambda r: "artist: " + r["artist_id"] + ", song: " + r["song_name"], axis=1)
song_artist_str = song_name_artist_ordered["artist_id"]

In [90]:
song_name_artist_ordered.shape

(3118, 3)

In [91]:
artists = song_name_artist_ordered["artist_id"].values

# Get colors for plot

In [92]:
famous = ["Lady GaGa", "Kanye West", "Bob Marley", "Bob Dylan", "Shakira", "Jay-Z", "Bon Jovi", "Michael Jackson", 
         "Mariah Carey", "Justin Timberlake", "Oasis"]

In [93]:
# colors = [int(a in famous) for a in artists]

In [94]:
# max_popularity = song_popularity.iloc[0].values[0]
# pre_colors = np.array([x[0]/max_popularity for x in song_popularity.values])

In [95]:
genres = list(set(track_genre.values()))
genres

['RnB',
 'Metal',
 'Rap',
 'Pop',
 'Rock',
 'Blues',
 'Latin',
 'World',
 'Country',
 'Jazz',
 'Vocal',
 'Electronic',
 'Pop_Rock',
 'International',
 'Reggae',
 'Folk',
 'New',
 'Punk']

In [96]:
genres.index("Blues")

5

In [97]:
def foo(t):
    global genres
    if t in track_genre:
        return genres.index(track_genre[t])
    return -1
colors = song_name_artist_ordered["track_id"].map(foo).values

In [98]:
colors

array([ 8,  4,  4, ...,  4, -1,  3])

In [99]:
for a in artists:
    if "Kurt" in a:
        print(a)

In [100]:
colors

array([ 8,  4,  4, ...,  4, -1,  3])

# Create t-SNE

In [103]:
filtered_song_ids

Index(['SOBONKR12A58A7A7E0', 'SOAUWYT12A81C206F1', 'SOSXLTC12AF72A7F54',
       'SOFRQTD12A81C233C0', 'SOEGIYH12A6D4FC0E3', 'SOAXGDH12A8C13F8A1',
       'SONYKOW12AB01849C9', 'SOPUCYA12A8C13A694', 'SOUFTBI12AB0183F65',
       'SOVDSJC12A58A7A271',
       ...
       'SOGRWNZ12A67ADEB27', 'SOESGET12AB0180504', 'SOSJAMD12AB017E3A6',
       'SOURVKJ12AB0189EF3', 'SOCMEJM12AF72A48D0', 'SONVHDR12A8C139B44',
       'SOFXJCS12A58A7DF02', 'SOWNIUS12A8C142815', 'SOAOQAM12A67ADE250',
       'SOIQFDB12A58A7E5F4'],
      dtype='object', name='song', length=3000)

In [104]:
song_factors = song_id_factors.loc[filtered_song_ids].values

In [105]:
 song_factors

array([[ 0.02682692, -0.05547712,  0.04097161, ..., -0.03776321,
        -0.05736425,  0.00342368],
       [ 0.02763558, -0.12945198, -0.05272642, ...,  0.01413184,
        -0.00717561, -0.10649522],
       [ 0.01602763, -0.16245782, -0.09897259, ..., -0.06576005,
        -0.01167816, -0.06561244],
       ...,
       [ 0.02466441, -0.06838492, -0.0234417 , ...,  0.05848533,
         0.05402588, -0.05095957],
       [-0.02402082,  0.00805999, -0.06100773, ..., -0.02550596,
        -0.03710075,  0.01029524],
       [-0.00304627, -0.06838337,  0.00048149, ...,  0.01881996,
         0.06498001,  0.06975536]])

In [106]:
song_factors.shape

(3000, 50)

In [115]:
id_count = {}
for i in song_name_artist_ordered.index:
    if i not in id_count:
        id_count[i] = 0
    id_count[i] += 1

In [118]:
imp = {}
for k, v in id_count.items():
    if v > 1:
        imp[k] = v

In [120]:
song_name_artist_ordered.loc[list(imp.keys())]

,track_id,artist_id,song_name
song_id,,,
SOGHSQA12A8C14098E,TRIMPFQ128F42B3024,Incubus,Dig
SOGHSQA12A8C14098E,TRTYUFJ128F92EDAA0,Incubus,Dig
SOJRFWQ12AB0183582,TRZIQMY128F146E70D,Fall Out Boy,Dance_ Dance
SOJRFWQ12AB0183582,TRJQKHL128F9304295,Fall Out Boy,Dance_ Dance
SOJRFWQ12AB0183582,TRYMVOW128F92ECE27,Fall Out Boy,Dance_ Dance
SORJVDO12AF72A1970,TRIBLJO128F932B6E9,Muse,Stockholm Syndrome
SORJVDO12AF72A1970,TRADYQK128F42A762A,Muse,Stockholm Syndrome
SOQTFOR12A6D4F612A,TRBWNTC128F9309A59,The Killers,All These Things That I've Done
SOQTFOR12A6D4F612A,TRNQGGL128F4214A9E,The Killers,All These Things That I've Done


In [108]:
X_embedded = TSNE(n_components=2).fit_transform(song_factors)
X_embedded.shape

(3000, 2)

In [109]:
by_genre_factors = {i: X_embedded[i == colors[:len(X_embedded)]] for i in range(len(genres))}
by_genre_text = {i: song_artist_str[i == colors] for i in range(len(genres))}

In [110]:
len(colors.keys())

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [121]:
# create a new plot with a title and axis labels
data = [go.Scatter(x=by_genre_factors[i][:,0], y=by_genre_factors[i][:,1], mode='text',
    marker=dict(
        size=7,
        colorscale='Jet',
    ), text=np.array(by_genre_text[i]), name=genres[i]) for i in range(len(genres))]

# fig = {
#     'data': [
#         {
#             'x': df[df['year']==year]['gdpPercap'],
#             'y': df[df['year']==year]['lifeExp'],
#             'name': year, 'mode': 'markers',
#         } for year in [1952, 1982, 2007]
#     ],
#     'layout': {
#         'xaxis': {'title': 'GDP per Capita', 'type': 'log'},
#         'yaxis': {'title': "Life Expectancy"}
#     }
# }


plotly.offline.iplot(data, filename="tsne-scatter")